### 3.6.1 MINST資料集

#### 載入資料集

In [1]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
import pickle      # 可以把執行程式的物件當作檔案儲存起來。當載入暫存的pickle檔案，就可以立刻回復剛才程式執行的物件。        
from dataset.mnist import load_mnist
from common.functions import sigmoid, softmax


#### 顯示資料維度

In [13]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=False, flatten=True, one_hot_label=False)
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


#### 顯示手寫字元影像

In [10]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import numpy as np
from dataset.mnist import load_mnist
from PIL import Image  # Python Image Library 

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

img = x_train[1]
label = t_train[1]
print(label)  # 5

print(img.shape)  # (784,) # 將一維的影像轉換回原本二維的影像 
img = img.reshape(28, 28)  # 形状を元の画像サイズに変形
print(img.shape)  # (28, 28)

img_show(img)


0
(784,)
(28, 28)


### 3.6.2 神經網路的(測試集)推論處理

In [12]:
# 測試資料集預測

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():       
    with open("sample_weight.pkl", 'rb') as f:  # 使用已訓練好的權重參數
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y


x, t = get_data()
network = init_network()
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i]) # 傳回值為經過softmax處理、加總為1的陣列
    p = np.argmax(y)           # 取得陣列中的最大值索引
    if p == t[i]:              # 一一比對
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

Accuracy:0.9352


### 3.6.3 批次處理

#### 顯示權重維度

In [15]:
x, _ = get_data()   
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

print('x : ', x.shape)
print('x[0] : ', x[0].shape)
print('W1 : ', W1.shape)
print('W2 : ', W2.shape)
print('W3 : ', W3.shape)

x :  (10000, 784)
x[0] :  (784,)
W1 :  (784, 50)
W2 :  (50, 100)
W3 :  (100, 10)


#### 加入批次處理

In [16]:
x, t = get_data()
network = init_network()

batch_size = 100 # バッチの数
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i: i+batch_size]                    # 0-100、100-200、... 
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)                  # 回傳各陣列中最大值的index，例如[1 2 5 9 1 3 0 ... 100th] 
    accuracy_cnt += np.sum(p == t[i:i+batch_size])  # 回傳布林振烈的加總，計算true的個數 

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))


Accuracy:0.9352


##### 批次運算註解

In [20]:
x = np.array([[0.1, 0.8, 0.1],
             [0.3, 0.1, 0.6],
             [0.2, 0.5, 0.3],
             [0.8, 0.1, 0.1]])
y = np.argmax(x, axis=1)          # 取得最大值的索引值
print(x.shape)
print(y)


(4, 3)
[1 2 1 0]


In [22]:
t = np.array([1, 2, 1, 0])
t = np.array([1, 2, 0, 0])
print(y==t)

np.sum(y==t)

[ True  True False  True]


3